In [3]:
import os
import glob

import pandas as pd
import numpy as np

# Preprocess ProteinGym Data

In [4]:

# Get all csv files in proteingym folder including subfolders
path = "/ConFit/data/proteingym"
csv_files = [f for f in glob.glob(path + "/**/*.csv", recursive=True)]
csv_files


# Get folder name
dsm_names = [os.path.basename(os.path.dirname(f)) for f in csv_files]
print(dsm_names)


['RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'RL40A_YEAST_Roscoe_2014', 'A0A2Z5U3Z0_9INFA_Wu_2014', 'A0A2Z5U3Z0_9INFA_Wu_2014', 'A0A2Z5U3Z0_9INFA_Wu_2014', 'A0A2Z5U3Z0_9INFA_Wu_2014', 'A0A2Z5U3Z0_9INFA_Wu_20

In [5]:
sel_dsm = "RASH_HUMAN_Bandaru_2017"

In [6]:
for csv_file in csv_files:
    # Only use data.csv files
    if ("data.csv" not in csv_file) and  (sel_dsm + ".csv" not in csv_file):
        continue
    print(csv_file)
    # Get folder name
    dsm_name = os.path.basename(os.path.dirname(csv_file))
    if dsm_name != sel_dsm or "data.csv" in csv_file or "wt.fasta" in csv_file:
        continue

    print(csv_file)

    orig_df = pd.read_csv(csv_file)
    orig_df["id"] = orig_df.index
    display(orig_df.head())

    # Rename some columns
    new_df = orig_df.rename(columns={"id": "PID", "DMS_score": "log_fitness", "mutated_sequence":"seq"})

    # Convert PID to int
    new_df["PID"] = new_df["PID"].astype(int)

    # Split the mutant column into multiple columns based on the ":"
    tmp_mut_cols = new_df["mutant"].str.split(":", expand=True).add_prefix("mutant_")
    new_df = pd.concat([new_df, tmp_mut_cols], axis=1)

    # Calculate the WT sequence from the first mutated sequence
    if "mutant_0" in new_df.columns:
        sel_col = "mutant_0"
    else:
        sel_col = "mutant"
    all_mut_cols = [col for col in new_df.columns if "mutant_" in col]

    # Get the mutant positions and amino acids for all mutants
    mut_df_1 = pd.DataFrame(columns=["mut_pos", "wt_aa", "mut_aa"])
    for sel_col in all_mut_cols:
        get_mut_pos = int(new_df[sel_col][0][1:-1])
        get_wt_aa = new_df[sel_col][0][0]
        get_mut_aa = new_df[sel_col][0][-1]
        sel_mut_df = pd.DataFrame({"mut_pos": get_mut_pos, "wt_aa": get_wt_aa, "mut_aa": get_mut_aa}, index=[0])
        mut_df_1 = pd.concat([mut_df_1, sel_mut_df], ignore_index=True)

    # Order the dataframe by mut_pos
    mut_df_1 = mut_df_1.sort_values(by="mut_pos")
    
    # Add a new WT row
    wt_row = new_df.iloc[0].copy()
    # Iterate over the mutations and add them to the WT sequence
    for i, row in mut_df_1.iterrows():
        get_mut_pos = row["mut_pos"]
        get_wt_aa = row["wt_aa"]
        wt_row["seq"] = wt_row["seq"][:get_mut_pos-1] + get_wt_aa + wt_row["seq"][get_mut_pos:]
    wt_row[all_mut_cols] = "WT"
    wt_row["mutant"] = "WT"
    wt_row["log_fitness"] = 1
    wt_row["performance"] = "-"
    # Add the WT row to the dataframe
    new_df = pd.concat([wt_row.to_frame().T, new_df], ignore_index=True)

    # Reset PID column
    new_df["PID"] = new_df.index

    # Find the number of mutations compared to the first sequence
    new_df["n_mut"] = new_df["seq"].apply(lambda x: sum(1 for a, b in zip(new_df["seq"][0], x) if a != b))
    new_df.head()

    # Get mutated positions
    new_df["mutated_position"] = new_df.apply(lambda x: [i for i in range(len(x["seq"])) if x["seq"][i] != new_df["seq"][0][i]], axis=1)
    # Convert to string separated by commas
    new_df["mutated_position"] = new_df["mutated_position"].apply(lambda x: ",".join(map(str, x)))
    new_df.head()

    # Assert if the number of mutations from the "mutant"  column is the same as the number of mutations
    # calculated from the sequence
    new_df["n_mut_2"] = new_df["mutant"].apply(lambda x: len(x.split(":")))
    # assert (far_df["n_mut"] == far_df["n_mut_2"]).all()
    new_df.head()

    # Replace ":" in the mutant column with ";"
    new_df["mutant"] = new_df["mutant"].str.replace(":", ";")

    # Find row with no mutations and write the seq to a fasta file named "wt.fasta"
    wt_seq = new_df[new_df["n_mut"] == 0]["seq"].values[0]
    with open(os.path.join(path, dsm_name,  "wt.fasta"), "w") as f:
        f.write(">wt\n")
        f.write(wt_seq)

    # Remove rows with no mutations
    new_df = new_df[new_df["n_mut"] > 0]

    # Write to file
    display(new_df.head())
    new_df.to_csv(os.path.join(path, dsm_name,  "data.csv"), index=False)
    
    # Copy folder to the /data folder
    os.system(f"cp -r {os.path.join(path, dsm_name)} /ConFit/data/")

/ConFit/data/proteingym/RL40A_YEAST_Roscoe_2014/data.csv
/ConFit/data/proteingym/A0A2Z5U3Z0_9INFA_Wu_2014/data.csv
/ConFit/data/proteingym/ENVZ_ECOLI_Ghose_2023/data.csv
/ConFit/data/proteingym/DYR_ECOLI_Thompson_2019/data.csv
/ConFit/data/proteingym/IF1_ECOLI_Kelsic_2016/data.csv
/ConFit/data/proteingym/GRB2_HUMAN_Faure_2021/data.csv
/ConFit/data/proteingym/A0A1I9GEU1_NEIME_Kennouche_2019/data.csv
/ConFit/data/proteingym/P84126_THETH_Chan_2017/data.csv
/ConFit/data/proteingym/RASH_HUMAN_Bandaru_2017/RASH_HUMAN_Bandaru_2017.csv
/ConFit/data/proteingym/RASH_HUMAN_Bandaru_2017/RASH_HUMAN_Bandaru_2017.csv


,mutant,mutated_sequence,DMS_score,DMS_score_bin,id
0,T2L,MLEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,-0.054586,1,0
1,T2C,MCEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,0.016885,1,1
2,T2D,MDEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,-0.080530,1,2
3,T2E,MEEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,-0.123798,1,3
4,T2F,MFEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,-0.030497,1,4


,mutant,seq,log_fitness,DMS_score_bin,PID,mutant_0,performance,n_mut,mutated_position,n_mut_2
1,T2L,MLEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,-0.054586,1,1,T2L,NaN,1,1,1
2,T2C,MCEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,0.016885,1,2,T2C,NaN,1,1,1
3,T2D,MDEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,-0.08053,1,3,T2D,NaN,1,1,1
4,T2E,MEEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,-0.123798,1,4,T2E,NaN,1,1,1
5,T2F,MFEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...,-0.030497,1,5,T2F,NaN,1,1,1


# Create train and test splits

In [7]:
def sample_data(data_dir, seed, shots, frac=0.5):
    '''
    sample the train data and test data
    :param seed: sample seed
    :param frac: the fraction of testing data, default to 0.2
    :param shot: the size of training data
    '''

    data = pd.read_csv(f"{data_dir}/data.csv", index_col=0)
    print(data.shape)
    if frac > 0:
        test_data = data.sample(frac=frac, random_state=seed)
        train_data = data.drop(test_data.index)
    elif frac == 0:
        # Shuffle the data
        data = data.sample(frac=1, random_state=seed)
        train_data = data
        test_data = pd.DataFrame()
    else:
        data = data.sample(frac=1, random_state=seed)
        train_data = data
        test_data = pd.DataFrame()
    print(train_data.shape)

    # Create a list of shots from a starting number and incrementing it until the training data is exhausted
    shots_list = np.arange(shots["starting_shots"], len(train_data), shots["increment"])
    
    for shot in shots_list:
        kshot_data = train_data.sample(n=shot, random_state=seed)
        assert len(kshot_data) == shot, (
            f'expected {shot} train examples, received {len(train_data)}')
        kshot_val_data = train_data.drop(kshot_data.index)
        kshot_data.to_csv(f"{data_dir}/train_{shot}_shot.csv")
        kshot_val_data.to_csv(f"{data_dir}/val_{shot}_shot.csv")
    else:
        train_data.to_csv(f"{data_dir}/train.csv")
        
    test_data.to_csv(f"{data_dir}/test.csv")


def split_train(data_dir, n_splits=5):
    '''
    five equal split training data, one of which will be used as validation set when training ConFit
    '''
    train = pd.read_csv(f"{data_dir}/train.csv", index_col=0)
    tlen = int(np.ceil(len(train) / n_splits))
    start = 0
    for i in range(1, n_splits):
        csv = train[start:start + tlen]
        start += tlen
        csv.to_csv(f"{data_dir}/train_{i}.csv")
    csv = train[start:]
    csv.to_csv(f"{data_dir}/train_{n_splits}.csv")

In [8]:
# Use the full data for training
protein_gym_dir = "/ConFit/data/proteingym/"
test_fraction = 0.5

# Dictionary for creating shots
shots = {}
shots["starting_shots"] = 48
shots["increment"] = 48

sample_data(os.path.join(protein_gym_dir,sel_dsm), seed=42, shots=shots, frac=test_fraction)

# Split the training data into 2 folds
# split_train(os.path.join(protein_gym_dir,sel_dsm), n_splits=2)




(3134, 9)
(1567, 9)
